# <font color=purple>Trying to understand how Word2Vec works</font>

In [ ]:
import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import logging 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [3]:
nltk.download('stopwords') #<-- we run this command to download the stopwords in the project
nltk.download('punkt') #<-- essential for tokenization
nltk.download('gutenberg') #<-- corpus for training the model

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [1]:
import re
import itertools
import math
import string
import nltk
# Access the Gutenberg Corpus
from nltk.corpus import gutenberg
import pandas as pd
import numpy as np

from nltk.probability import FreqDist

from nltk.corpus import stopwords

# Function from lecturer
## <font color=purple>The function that cleans the data</font>

In [27]:
def clean(inp: str) -> str:

    inp = inp.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))
    inp = re.sub(r'\s+', ' ', inp.lower())

    return inp

In [208]:
# from a list of sentences
documents = ["If you use a car frequently, the first step to cutting",
             "down your emissions may well be to simply", 
             "fully consider the", 
             "alternatives available to you."
             ]
clean(str(documents))

' if you use a car frequently the first step to cutting down your emissions may well be to simply fully consider the alternatives available to you '

# <font color=purple>To train my own Word2Vec model I need:</font>
1. **Obtain a large corpus of text**: You will need a large amount of text data to train your Word2vec model. You can obtain text data from various sources such as Wikipedia, news articles, or social media posts.
2. **Preprocess the text**: Before training the Word2vec model, you will need to preprocess the text data by removing stop words, punctuations, and other non-essential elements. You can also tokenize the text data into words or phrases to prepare it for training.
3. **Choose a Word2vec algorithm**: There are two main algorithms for training Word2vec models: Continuous Bag of Words (CBOW) and Skip-gram. CBOW predicts a word based on its context, while Skip-gram predicts the context based on a word. Choose the algorithm that best suits your needs.
4. **Train the Word2vec model**: You can train the Word2vec model using popular libraries such as Gensim or TensorFlow. These libraries provide easy-to-use functions to train and test the Word2vec model. During training, the model learns to associate each word in the vocabulary with a vector of real numbers, which represent the word embedding.
5. **Evaluate the Word2vec model**: Once the Word2vec model is trained, you can evaluate its performance using intrinsic or extrinsic evaluation methods. Intrinsic evaluation focuses on evaluating specific aspects of the model in isolation, such as its ability to generate embeddings or to classify sentences based on sentiment. Extrinsic evaluation measures the performance of the model on a specific task, such as language modeling or sentiment analysis.
6. **Use the Word2vec model**: After training and evaluating the Word2vec model, you can use it for various natural language processing tasks, such as information retrieval, text classification, or machine translation.

## <font color=blue>Obtain a large corpus of text</font>
> Попробуем сначала поработать с корпусом Gutenberg (классическая литература), если что, переключимся на Brown (новости на английском)

In [2]:
# Print the categories in the Brown Corpus
print(gutenberg.fileids(), '\n')

# Print the first few sentences of a book
sentences = gutenberg.sents('carroll-alice.txt')
for sentence in sentences[:5]:
    print(sentence)

# Transform data from nested lists to one list
result1 = list(itertools.chain(*sentences))
print(result1[:20])

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt'] 

['[', 'Alice', "'", 's', 'Adventures', 'in', 'Wonderland', 'by', 'Lewis', 'Carroll', '1865', ']']
['CHAPTER', 'I', '.']
['Down', 'the', 'Rabbit', '-', 'Hole']
['Alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', 'bank', ',', 'and', 'of', 'having', 'nothing', 'to', 'do', ':', 'once', 'or', 'twice', 'she', 'had', 'peeped', 'into', 'the', 'book', 'her', 'sister', 'was', 'reading', ',', 'but', 'it', 'had', 'no', 'pictures', 'or', 'conversations', 'in', 'it', ',', "'", 'and', 'what', 'is', 'the', 'use', 'of', 'a', 'bo

## <font color=blue>Preprocess the text</font>

Source: https://towardsdatascience.com/how-to-train-a-word2vec-model-from-scratch-with-gensim-c457d587e031

In [275]:
def preprocess_text(text: str) -> str:
    """Function that cleans the input text by going to:
    - remove links
    - remove special characters
    - remove numbers
    - remove stopwords
    - convert to lowercase
    - remove excessive white spaces
    Arguments:
        text (str): text to clean
        remove_stopwords (bool): whether to remove stopwords
    Returns:
        str: cleaned text
    """
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove numbers and special characters
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    # 1. create tokens
    tokens = nltk.word_tokenize(text)
    # 2. check if it's a stopword
    tokens = [w.lower().strip() for w in tokens if not w.lower() in stopwords.words("english")]
    # return a list of cleaned tokens
    return [tokens]

In [276]:
texts = preprocess_text(clean(str(documents)))
texts

[['use',
  'car',
  'frequently',
  'first',
  'step',
  'cutting',
  'emissions',
  'may',
  'well',
  'simply',
  'fully',
  'consider',
  'alternatives',
  'available']]

## <font color=blue>Train the Word2Vec model</font>

## <font color=blue>Convert to one function</font>

In [282]:
import string

def tokenize(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Split text into tokens (words)
    tokens = text.split()
    return [tokens]

In [247]:
def build_vocab(sentences):
    words = []
    for sentence in sentences:
        words.extend(sentence)
    words = sorted(set(words))
    word2id = {w: i for i, w in enumerate(words)}
    id2word = {i: w for w, i in word2id.items()}
    return word2id, id2word

In [157]:
def skipgrams(sentence, window_size):
    pairs = []
    for i, w in enumerate(sentence):
        for j in range(i - window_size, i + window_size + 1):
            if j != i and j >= 0 and j < len(sentence):
                pairs.append((w, sentence[j]))
    return pairs

In [158]:
def initialize_weights(vocabulary_size, vector_size):
    W1 = np.random.uniform(-0.5, 0.5, size=(vocabulary_size, vector_size))
    W2 = np.random.uniform(-0.5, 0.5, size=(vocabulary_size, vector_size))
    return W1, W2

In [286]:
def pred_function(x):
    exp_scores = np.exp(x - np.max(x))
    return exp_scores / np.sum(exp_scores, axis=0)

In [160]:
def update_weights(W1, W2, target_word_idx, context_words, dL_dh, dL_du, learning_rate):
    W2 -= learning_rate * np.outer(W1[target_word_idx], dL_du)
    W1[target_word_idx] -= learning_rate * dL_dh

In [205]:
def train_old(pairs, word2id, W1, W2, learning_rate, epochs, vector_size):
    for epoch in range(epochs):
        loss_history = []
        loss = 0
        for pair in pairs:
            center_word = np.zeros((vector_size,))
            context_word = np.zeros((vector_size,))
            u = word2id[pair[0]]
            v = word2id[pair[1]]
            center_word = W1[u]
            for context in range(len(word2id)):
                if context == v:
                    context_word = W2[context]
                    z = np.dot(center_word, context_word)
                    sig = pred_function(z)
                    e = (1 - int(context == v)) - sig
                    loss += e**2
                    grad_sig = e * learning_rate
                    grad_context_word = center_word * grad_sig
                    grad_center_word = context_word * grad_sig
                    W1[u] -= grad_center_word
                    W2[context] -= grad_context_word
        # loss_history.append(loss / len(pairs))
    return W1, W2

In [287]:
def train(data: str):
    # All parameters for training the Word2Vec model
    window_size=2
    vector_size=10
    learning_rate=0.025
    epochs=10

    # Create tokens
    text = tokenize(data)

    # Build vocabulary
    word2id, id2word = build_vocab(text)

    # Generate skip-grams
    pairs = []
    for sentence in data:
        pairs.extend(skipgrams(sentence, window_size))

    # Initialize weights
    W, W_context = initialize_weights(len(word2id), vector_size)

    # Train model
    W, W_context = train_old(pairs, word2id, W, W_context, learning_rate, epochs, vector_size)

    # Create final dictionary
    dict_final = {key: W[word2id[key]] for key in word2id.keys()}

    return dict_final

In [288]:
train(clean(str(documents)))

{'a': array([ 0.28847533, -0.42579127, -0.49083777,  0.39821145,  0.23358213,
         0.10700838, -0.17352341, -0.38870176,  0.38830678, -0.46624828]),
 'alternatives': array([-0.00880526,  0.27288321, -0.07149046,  0.10515103,  0.44149732,
        -0.1644551 ,  0.25820854,  0.30934356, -0.19587371, -0.1658254 ]),
 'available': array([-0.45579346, -0.21392973, -0.10299653, -0.45210641, -0.21152857,
         0.09706983, -0.39095764,  0.36747191,  0.46069989, -0.1678274 ]),
 'be': array([-0.07733037, -0.20770594, -0.1447155 , -0.28522312, -0.29327728,
        -0.11703954, -0.38305983, -0.2406282 , -0.14250838, -0.46011462]),
 'car': array([-0.23785869,  0.18214342, -0.40325524, -0.07496411, -0.43008292,
        -0.04985569, -0.16676924, -0.29822325, -0.32330982,  0.11061734]),
 'consider': array([-0.10587109, -0.24405389,  0.08443648, -0.08020798,  0.37298265,
         0.3366705 ,  0.49947678, -0.02671674,  0.18346326, -0.29558107]),
 'cutting': array([-0.20145565, -0.38432145, -0.18997

In [283]:
tokenize(clean(str(documents)))

[['if',
  'you',
  'use',
  'a',
  'car',
  'frequently',
  'the',
  'first',
  'step',
  'to',
  'cutting',
  'down',
  'your',
  'emissions',
  'may',
  'well',
  'be',
  'to',
  'simply',
  'fully',
  'consider',
  'the',
  'alternatives',
  'available',
  'to',
  'you']]

---
Embedings are not good enouth

In [302]:
import numpy as np
import string

def tokenize(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Split text into tokens (words)
    tokens = text.split()
    return [tokens]

def build_vocab(sentences):
    words = []
    for sentence in sentences:
        words.extend(sentence)
    words = sorted(set(words))
    word2id = {w: i for i, w in enumerate(words)}
    id2word = {i: w for w, i in word2id.items()}
    return word2id, id2word
    
def skipgrams(sentence, window_size):
    pairs = []
    for i, w in enumerate(sentence):
        for j in range(i - window_size, i + window_size + 1):
            if j != i and j >= 0 and j < len(sentence):
                pairs.append((w, sentence[j]))
    return pairs

def initialize_weights(vocabulary_size, vector_size):
    W1 = np.random.uniform(-0.5, 0.5, size=(vocabulary_size, vector_size))
    W2 = np.random.uniform(-0.5, 0.5, size=(vocabulary_size, vector_size))
    return W1, W2

def pred_function(x):
    exp_scores = np.exp(x - np.max(x))
    return exp_scores / np.sum(exp_scores, axis=0)
    
def update_weights(W1, W2, target_word_idx, context_words, dL_dh, dL_du, learning_rate):
    W2 -= learning_rate * np.outer(W1[target_word_idx], dL_du)
    W1[target_word_idx] -= learning_rate * dL_dh

def train_old(pairs, word2id, W1, W2, learning_rate, epochs, vector_size):
    for epoch in range(epochs):
        loss_history = []
        loss = 0
        for pair in pairs:
            center_word = np.zeros((vector_size,))
            context_word = np.zeros((vector_size,))
            u = word2id[pair[0]]
            v = word2id[pair[1]]
            center_word = W1[u]
            for context in range(len(word2id)):
                if context == v:
                    context_word = W2[context]
                    z = np.dot(center_word, context_word)
                    sig = pred_function(z)
                    e = (1 - int(context == v)) - sig
                    loss += e**2
                    grad_sig = e * learning_rate
                    grad_context_word = center_word * grad_sig
                    grad_center_word = context_word * grad_sig
                    W1[u] -= grad_center_word
                    W2[context] -= grad_context_word
        # loss_history.append(loss / len(pairs))
    return W1, W2


def train(data: str):
    # All parameters for training the Word2Vec model
    window_size=3
    vector_size=10
    learning_rate=0.001
    epochs=100

    # Create tokens
    text = tokenize(data)

    # Build vocabulary
    word2id, id2word = build_vocab(text)

    # Generate skip-grams
    pairs = []
    for sentence in data:
        pairs.extend(skipgrams(sentence, window_size))

    # Initialize weights
    W, W_context = initialize_weights(len(word2id), vector_size)

    # Train model
    W, W_context = train_old(pairs, word2id, W, W_context, learning_rate, epochs, vector_size)

    # Create final dictionary
    dict_final = {key: W[word2id[key]] for key in word2id.keys()}

    return dict_final

In [303]:
train(clean(str(documents)))

{'a': array([-0.36776425,  0.16565639,  0.04372147,  0.06616889,  0.38856678,
        -0.21054818, -0.45188995,  0.43750588,  0.4791722 , -0.03311668]),
 'alternatives': array([ 0.32914017,  0.19904258, -0.3195285 ,  0.14057175, -0.4139773 ,
        -0.03157049,  0.13688819, -0.18928625,  0.15975498,  0.14356051]),
 'available': array([-0.33941646,  0.07338916,  0.42723351, -0.42056585, -0.45153616,
        -0.49481611,  0.19764023,  0.00408344,  0.4252313 , -0.08517067]),
 'be': array([ 0.29592341,  0.34558949,  0.13111804, -0.44441846, -0.48131925,
         0.19061378, -0.23116951, -0.45149755, -0.29856515,  0.11739898]),
 'car': array([ 0.44380004, -0.13956756, -0.38965916,  0.03603478,  0.27824233,
         0.0778963 ,  0.39890429, -0.39173743, -0.25234295,  0.0020075 ]),
 'consider': array([ 0.44130905,  0.01574036,  0.41146182, -0.40065405,  0.21781645,
        -0.03009246, -0.08709338, -0.0534225 ,  0.21861278, -0.47479319]),
 'cutting': array([ 0.03926995, -0.3398514 ,  0.06923